In [ ]:
import pandas as pd
from glob import glob
from random import choice, sample
import altair as alt
from os import exists

In [ ]:
country_filepaths = glob("*/*/*/*_data.csv")
print(country_filepaths)

## Testing With One Country

In [ ]:
poland = pd.read_csv(country_filepaths[0])

In [ ]:
poland.head()

In [ ]:
poland.info()

In [ ]:
poland_long = poland.melt(
        id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
        var_name='Year', value_name='Value'
    )

In [ ]:
poland_long[poland_long["Value"].notna()].head().max()

In [ ]:
pivoted_df = poland_long.pivot_table(
    index=['Country Name', 'Country Code', 'Year'],
    columns='Indicator Name',
    values='Value'
)

## Combining All Countries

Data is shaped as follows:
> [Country, Country Code, Year, [All Indicators]]

In [ ]:
df_list = []

In [ ]:
for country in country_filepaths:
    df = pd.read_csv(country)
    df_long = df.melt(
        id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
        var_name='Year', value_name='Value'
    )
    pivoted_df = df_long.pivot_table(
    index=['Country Name', 'Country Code', 'Year'],
    columns='Indicator Name',
    values='Value'
    ).reset_index()
    df_list.append(pivoted_df)

In [ ]:
combined_data = pd.concat(df_list)

In [ ]:
combined_data.dropna(subset="Year", inplace=True)
combined_data = combined_data.astype({"Year": int})

In [ ]:
for col in combined_data.columns:
    if combined_data[col].dtype != float:
        print(col, combined_data[col].dtype)

In [ ]:
cols = ['Country Name', 'Country Code', 'Year'] + [col for col in combined_data if col not in ['Country Name', 'Country Code', 'Year', "Indicator Name"]]
combined_data = combined_data[cols].reset_index()

In [ ]:
combined_data.sample(10)

## Choosing a random Indicator to Check For 3 Random Countries

In [ ]:
randomly_chosen_countries = sample(combined_data['Country Name'].unique().tolist(), 7)
filtered_data = combined_data[combined_data['Country Name'].isin(randomly_chosen_countries)]
randomly_chosen_countries


In [ ]:
randomly_chosen_column = choice([col for col in combined_data if col not in ['Country Name', 'Country Code', 'Year', "Indicator Name"]])
randomly_chosen_column

In [ ]:
alt.Chart(filtered_data).mark_line().encode(
    x='Year:O',
    y=alt.Y(randomly_chosen_column.replace(":", r"\:"), title=f'{randomly_chosen_column}'),
    color="Country Name"
).properties(
    title=f'{randomly_chosen_column} by Year'
).show()

In [ ]:
if not exists(f"{randomly_chosen_column} by Year".replace(" ", "_")):
    ...

In [ ]:
combined_data.to_csv("./data/world_bank/combined_world_data.csv", index=False)